## Bert用于文本分类任务的应用步骤
1. 预训练阶段<br>
Bert通过大规模的无标签文本数据进行预训练，学习了丰富的语言表示
* 掩码语言模型（Masked Language Model，MLM）：随机遮掩输入文本中的一些单词，模型的任务就是预测被遮掩的单词，这使得模型能够理解上下文信息；
* 下一个句子预测（Next Sentence Prediction，NSP）：输入两个句子，模型需要判断第二个句子是否是第一个句子的后续。

2. 微调阶段<br>
在预训练之后，可以通过将Bert微调来适应特定的文本分类任务
* 数据准备：准备好带标签的数据集，包括文本及其对应的分类标签
* 模型构建：使用`BertForSequenceClassification`等类加载预训练的Bert模型，并提那家一个分类层（全连接层）来输出分类结果
* 输入格式化：将文本数据转换为Bert可以接受的格式
    * Tokenization：使用Bert的分词器将文本转换为token ID，并进行填充
    * Attention Mask：生成一个Mask，指示模型应该关注哪些token（1表示有效token，0表示填充token）

3. 训练<br>
在微调过程中，通过使用带标签的训练数据对模型进行训练。模型会根据输入文本和相应的标签调整其权重，以最大化正确分类的概率。使用损失函数（如交叉熵损失）来评估模型性能。

4. 评估与预测<br>
在训练完成后，可以使用验证集或测试集对模型进行评估，以确定其分类性能。模型可以输出每个类别的概率分布，用户可以根据最大概率进行类别判断。

### 1. 导包

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# BertForSequenceClassification：Hugging Face transformers库中的一个类，专门用于处理序列分类任务，例如情感分析、主题分类等
from datasets import load_dataset
import torch

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bytedance/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. 加载预训练的BERT模型和分词器

In [4]:
model_name = "bert-base-uncased"  # 可以根据需要选择不同的BERT版本
tokenizer = BertTokenizer.from_pretrained(model_name) # 分词器
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 适用于二分类
# :param model_name: 指定要加载的预训练BERT模型的名称或者路径
# :param num_labels: 指定模型的输出标签数量

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 3. 加载数据集

In [5]:
# 加载数据集，这里使用的是 Hugging Face 自带的 dataset 库
dataset = load_dataset("imdb")  # 示例是IMDB影评分类任务

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [6]:
df_train=dataset['train'].to_pandas()
print(df_train.head())

                                                text  label
0  I rented I AM CURIOUS-YELLOW from my video sto...      0
1  "I Am Curious: Yellow" is a risible and preten...      0
2  If only to avoid making this type of film in t...      0
3  This film was probably inspired by Godard's Ma...      0
4  Oh, brother...after hearing about this ridicul...      0


In [7]:
df_test=dataset['train'].to_pandas()
print(df_test.head())

                                                text  label
0  I rented I AM CURIOUS-YELLOW from my video sto...      0
1  "I Am Curious: Yellow" is a risible and preten...      0
2  If only to avoid making this type of film in t...      0
3  This film was probably inspired by Godard's Ma...      0
4  Oh, brother...after hearing about this ridicul...      0


### 4. 定义数据预处理函数

In [4]:
# 数据预处理函数，将文本转化为模型输入格式
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

### 5. 数据集预处理

In [5]:
# 对数据集进行分词和编码
encoded_dataset = dataset.map(preprocess_function, batched=True)

### 6. 模型定义与编译

In [6]:
# 从数据集中抽取一部分样本用于debug模式
small_train_dataset = encoded_dataset['train'].select(range(100))  # 抽取100个样本作为训练集
small_eval_dataset = encoded_dataset['test'].select(range(100))    # 抽取100个样本作为验证集


In [7]:
# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',          # 输出结果保存的路径
    evaluation_strategy="epoch",     # 在每个epoch后进行验证
    per_device_train_batch_size=16,  # 训练批量大小
    per_device_eval_batch_size=64,   # 验证批量大小
    num_train_epochs=3,              # 训练的epoch数
    weight_decay=0.01,               # 权重衰减
)

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
# 使用Trainer API来进行训练和验证
trainer = Trainer(
    model=model,                         # 要训练的模型
    args=training_args,                  # 训练参数
    train_dataset=small_train_dataset,  # 训练集
    eval_dataset=small_eval_dataset,    # 验证集
)

In [9]:
# 开始训练
trainer.train()

 33%|███▎      | 7/21 [00:27<00:32,  2.34s/it]

{'eval_loss': 0.09301885962486267, 'eval_runtime': 4.9323, 'eval_samples_per_second': 20.275, 'eval_steps_per_second': 0.405, 'epoch': 1.0}


 67%|██████▋   | 14/21 [00:46<00:13,  1.86s/it]

{'eval_loss': 0.028304338455200195, 'eval_runtime': 4.9948, 'eval_samples_per_second': 20.021, 'eval_steps_per_second': 0.4, 'epoch': 2.0}


                                               
100%|██████████| 21/21 [01:09<00:00,  3.30s/it]

{'eval_loss': 0.01811107248067856, 'eval_runtime': 6.7211, 'eval_samples_per_second': 14.879, 'eval_steps_per_second': 0.298, 'epoch': 3.0}
{'train_runtime': 69.3774, 'train_samples_per_second': 4.324, 'train_steps_per_second': 0.303, 'train_loss': 0.13575719651721774, 'epoch': 3.0}


TrainOutput(global_step=21, training_loss=0.13575719651721774, metrics={'train_runtime': 69.3774, 'train_samples_per_second': 4.324, 'train_steps_per_second': 0.303, 'total_flos': 78933316608000.0, 'train_loss': 0.13575719651721774, 'epoch': 3.0})

In [10]:
# 保存模型
model.save_pretrained('./bert_text_classification')
tokenizer.save_pretrained('./bert_text_classification')

('./bert_text_classification/tokenizer_config.json',
 './bert_text_classification/special_tokens_map.json',
 './bert_text_classification/vocab.txt',
 './bert_text_classification/added_tokens.json')